In [1]:
%load_ext autoreload
%autoreload 2
#%reload_ext autoreload

In [1]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import Generator as Generator 
import model_definitions

### Open input file

In [2]:
SEQUENCE_LENGTH=20
ONE_CAR=True
PATH=r"C:\Users\dansp\OneDrive\Desktop\Tesi\Logs\OneCar_db.txt"
#PATH=r"C:\Users\dansp\OneDrive\Desktop\Tesi\SCRITTURA\OneCar_tack1_db.txt"
BATCH_SIZE=256*2
FEATURES=7 
FEATURES_PREDICTED=3
DISCARD=9

In [3]:
maxSpeed=40
frequency=0.02
maxdiff=maxSpeed*frequency
maxAngularVelocity=140 #140 degrees for second
maxAngDiff=maxAngularVelocity*frequency
print(f"Max speed diff: {maxdiff}\nMax x/z diff: {maxdiff}\nMax rot diff: {maxAngDiff}")

Max speed diff: 0.8
Max x/z diff: 0.8
Max rot diff: 2.8000000000000003


In [4]:
Generator.SEQUENCE_LENGTH=SEQUENCE_LENGTH
Generator.FEATURES=FEATURES
Generator.DISCARD=DISCARD
train,val,test=Generator.single_care_dataframe(PATH)

In [5]:
def SubtractDF(df):
    return df.groupby("RACE",group_keys=False).apply(Generator.subtraction_columns)

def SubtractAndBuildDF(df):
    dfs=[]
    for i in range(len(df)):
        dfs.append(SubtractDF(df[i]))
        #dfs.append(df[i])
    united=dfs[0]
    for i in range(1,len(df)):
        united=pd.concat([united,dfs[i] ],ignore_index=True)
    return united

df_train= SubtractAndBuildDF(train)
df_val= SubtractAndBuildDF(val)
df_test= SubtractAndBuildDF(test)

In [6]:
df_train.head()

,X,Z,VEL_X,VEL_Z,ROT,ANG_VEL_Y,ACC_X,ACC_Z,TILE,TILE_IND,X_RELATIVE,Z_RELATIVE,TIME,RACE,GROUP
0,0.00000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,4,32,0.000000,0.000000,0.2,0,0
1,-0.00066,0.39131,-0.030470,3.676403,-0.0278,-0.024275,-1.523513,16.220290,4,32,-0.124009,0.394551,0.2,0,0
2,-0.20325,0.98584,-0.372497,2.119079,-5.7426,-0.118653,35.271613,-13.825137,4,32,-0.126719,0.407695,0.2,0,0
3,-0.27722,1.33862,-0.699331,1.527281,-3.7373,0.280035,-21.201270,1.749897,4,32,-0.130415,0.425544,0.2,0,0
4,-0.19885,1.60254,-0.089037,1.494527,1.7791,-0.131975,-14.416056,6.210950,4,32,-0.133066,0.446911,0.2,0,0


In [63]:
#df_train,df_val,df_test= Generator.split_train_validation_test(united,["RACE","GROUP"])

In [64]:
#df_train.head()

,X,Z,VEL_X,VEL_Z,ROT,ANG_VEL_Y,ACC_X,ACC_Z,TILE,TILE_IND,X_RELATIVE,Z_RELATIVE,TIME,RACE,GROUP
0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0,0,0.000000,0.000000,0.0,0,0
1,-0.0002,0.436992,-0.000143,4.093906,-0.002010,-0.000009,-0.000736,19.05289,16,0,0.221663,-0.219433,0.2,0,0
2,0.0000,1.183716,-0.000108,3.237970,-0.000377,-0.000379,0.000330,-1.26555,16,0,0.221663,-0.179976,0.2,0,0
3,0.0000,1.748169,-0.001402,2.559089,-0.000021,0.021195,-0.182380,-1.23761,16,0,0.221663,-0.121704,0.2,0,0
4,0.0000,2.215698,0.002161,2.152595,-0.004692,-0.038634,0.279078,-1.00589,16,0,0.221663,-0.047847,0.2,0,0


In [7]:
gen_train=Generator.DataGenerator(BATCH_SIZE,df_train,100000)
gen_val=Generator.DataGenerator(BATCH_SIZE,df_val,100000)
gen_test=Generator.DataGenerator(BATCH_SIZE,df_test,100000)


Length: 2720316 races: 84 n batches: 2713596 / 512
Length: 1286892 races: 42 n batches: 1283532 / 512
Length: 1303610 races: 42 n batches: 1300250 / 512


In [29]:
#x_y=gen_test[0]
#print(x_y[0][0])
#x_y[1][0]


tf.Tensor(
[[ 3.4070000e-01 -1.4206130e+00 -2.1693000e+00  1.9000000e+01
   8.0000000e+00 -1.4785160e-01 -2.9850260e-02]
 [ 2.6844100e+00 -5.7081530e+00 -1.1614300e+01  1.0000000e+00
   9.0000000e+00  4.7454430e-01 -3.8924150e-02]
 [ 4.9586100e+00 -7.2426800e+00 -1.1956000e+01  1.0000000e+00
   9.0000000e+00  3.6210940e-01 -8.7801110e-02]
 [ 2.6165300e+00 -3.4274800e+00 -7.3431000e+00  1.0000000e+00
   9.0000000e+00  2.6189780e-01 -1.5257160e-01]
 [ 2.2347400e+00 -2.4335800e+00 -4.1457000e+00  1.0000000e+00
   9.0000000e+00  1.6808270e-01 -2.2641600e-01]
 [ 4.6888900e+00 -3.0961200e+00 -1.0811800e+01  1.0000000e+00
   9.0000000e+00  8.6621090e-02 -3.0963130e-01]
 [ 5.4751900e+00 -2.4381900e+00 -9.6003000e+00  1.0000000e+00
   9.0000000e+00  2.2639970e-02 -4.0230310e-01]
 [ 5.9890100e+00 -1.3954000e+00 -1.0225400e+01  1.7000000e+01
   1.0000000e+01 -2.2705080e-02  4.9762370e-01]
 [ 3.1513150e+00 -6.6466000e-01 -6.0329000e+00  1.7000000e+01
   1.0000000e+01 -5.2343750e-02  2.9331060e-01]

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 0.432777, -0.38731 , -0.9811  ])>

In [30]:
#x_y[0][0]

<tf.Tensor: shape=(20, 7), dtype=float64, numpy=
array([[ 3.4070000e-01, -1.4206130e+00, -2.1693000e+00,  1.9000000e+01,
         8.0000000e+00, -1.4785160e-01, -2.9850260e-02],
       [ 2.6844100e+00, -5.7081530e+00, -1.1614300e+01,  1.0000000e+00,
         9.0000000e+00,  4.7454430e-01, -3.8924150e-02],
       [ 4.9586100e+00, -7.2426800e+00, -1.1956000e+01,  1.0000000e+00,
         9.0000000e+00,  3.6210940e-01, -8.7801110e-02],
       [ 2.6165300e+00, -3.4274800e+00, -7.3431000e+00,  1.0000000e+00,
         9.0000000e+00,  2.6189780e-01, -1.5257160e-01],
       [ 2.2347400e+00, -2.4335800e+00, -4.1457000e+00,  1.0000000e+00,
         9.0000000e+00,  1.6808270e-01, -2.2641600e-01],
       [ 4.6888900e+00, -3.0961200e+00, -1.0811800e+01,  1.0000000e+00,
         9.0000000e+00,  8.6621090e-02, -3.0963130e-01],
       [ 5.4751900e+00, -2.4381900e+00, -9.6003000e+00,  1.0000000e+00,
         9.0000000e+00,  2.2639970e-02, -4.0230310e-01],
       [ 5.9890100e+00, -1.3954000e+00, -1.02254

### Model definition

#### Train and print

In [87]:
if(ONE_CAR):
    model_definitions.cars=1
else:
    model_definitions.cars=4
model_definitions.features=FEATURES
model_definitions.sequence_length=SEQUENCE_LENGTH
model_definitions.feat_pred=FEATURES_PREDICTED

### Rules Model

In [ ]:
lstm,history=LSTM_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.01,epoch=100, dropout=0.2, cells=6 )

ev=lstm.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
lstm2,history=LSTM_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.002,epoch=100, dropout=0.02, cells=1)

ev=lstm2.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
mlp,history=MLP_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.001,epoch=100)

ev=mlp.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
siamese,history=LSTM_Siamese_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.001,epoch=100, dropout=0.02, cells=2 )

ev=siamese.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

### Physichs Model

Since physichs data can be very small, Each data is multiply by 10 

In [14]:
NAME=f"MLP_DISCARD_{DISCARD}_DELTA_LONG_TRAIN"
mlp_ph,history_mlp_ph=model_definitions.MLP_Train(gen_train,gen_val,learning_rate=0.0001,epoch=1000,verbose=1,name=NAME,path="logs")

ev=mlp_ph.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")
tf.saved_model.save(mlp_ph,f'saved_model/{NAME}')

Epoch 1/10
5158/5158 [==============================] - 1237s 240ms/step - loss: 12.4075 - mean_absolute_error: 1.3992 - accuracy: 0.0000e+00 - val_loss: 1.7049 - val_mean_absolute_error: 0.8486 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 2/10
5158/5158 [==============================] - 1758s 341ms/step - loss: 1.3769 - mean_absolute_error: 0.7643 - accuracy: 0.0000e+00 - val_loss: 1.2612 - val_mean_absolute_error: 0.7272 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 3/10
5158/5158 [==============================] - 1663s 322ms/step - loss: 1.2098 - mean_absolute_error: 0.7139 - accuracy: 0.0000e+00 - val_loss: 1.2066 - val_mean_absolute_error: 0.7096 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 4/10
5158/5158 [==============================] - 1689s 327ms/step - loss: 1.1864 - mean_absolute_error: 0.7058 - accuracy: 0.0000e+00 - val_loss: 1.1945 - val_mean_absolute_error: 0.7048 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 5/10
5158/5158 [=========================

In [13]:
units=32
kernel_size=1
filters=32
normalization=False
cnn_lstm,history_cnn_lstm=model_definitions.CNN_LSTM_Train(gen_train,gen_val,learning_rate=0.0001,patience=10,name=f"CNN{filters}_{kernel_size}LSTM{units}_Norm{normalization}",
                                                           path="logs_350SequenceLength",epoch=10, dropout=0.2,units=units,kernel_size=kernel_size,
                                                           filters=filters, normalization=normalization,verbose=1
                                                          )

ev=cnn_lstm.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

tf.saved_model.save(cnn_lstm,f'saved_model/CNN_LSTM{units}_{kernel_size}_{filters}_PosAssoluta')

Epoch 1/10
5655/5655 [==============================] - 2340s 413ms/step - loss: 0.9373 - mean_absolute_error: 0.6307 - accuracy: 4.6051e-08 - val_loss: 0.7536 - val_mean_absolute_error: 0.5764 - val_accuracy: 9.2807e-08 - lr: 1.0000e-04
Epoch 2/10
5655/5655 [==============================] - 2301s 407ms/step - loss: 0.5683 - mean_absolute_error: 0.5047 - accuracy: 0.0000e+00 - val_loss: 0.4222 - val_mean_absolute_error: 0.4416 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 3/10
5655/5655 [==============================] - 2333s 412ms/step - loss: 0.3508 - mean_absolute_error: 0.4066 - accuracy: 0.0000e+00 - val_loss: 0.2898 - val_mean_absolute_error: 0.3652 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 4/10
5655/5655 [==============================] - 2304s 407ms/step - loss: 0.2785 - mean_absolute_error: 0.3564 - accuracy: 0.0000e+00 - val_loss: 0.2501 - val_mean_absolute_error: 0.3341 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 5/10
5655/5655 [==========================

INFO:tensorflow:Assets written to: saved_model/CNN_LSTM32_1_32_PosAssoluta\assets


INFO:tensorflow:Assets written to: saved_model/CNN_LSTM32_1_32_PosAssoluta\assets


In [88]:
lr =1e-4
units=64
cells=3
normalization=False
name=f"LSTM{cells}_{units}_{lr}_ROT_NO_VEL_INPUT_TRACK1_{DISCARD}"
lstm_ph,history_lstm_ph=model_definitions.LSTM_Train(
    gen_train,gen_val,learning_rate=lr,patience=6,name=name,path="track1_logs",
    epoch=20, dropout=0.2, cells=cells,units=units, normalization=normalization,verbose=1
)

ev=lstm_ph.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")
#tf.saved_model.save(lstm_ph,f'saved_model/{name}')

Epoch 1/20
98/98 [==============================] - 37s 293ms/step - loss: 6.4422 - mean_absolute_error: 1.4853 - accuracy: 0.0000e+00 - val_loss: 6.2958 - val_mean_absolute_error: 1.4331 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 2/20
98/98 [==============================] - 27s 276ms/step - loss: 4.3596 - mean_absolute_error: 1.2809 - accuracy: 0.0000e+00 - val_loss: 4.4452 - val_mean_absolute_error: 1.3055 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 3/20
98/98 [==============================] - 27s 272ms/step - loss: 3.4043 - mean_absolute_error: 1.1292 - accuracy: 0.0000e+00 - val_loss: 3.1030 - val_mean_absolute_error: 1.0968 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 4/20
98/98 [==============================] - 27s 278ms/step - loss: 2.8065 - mean_absolute_error: 1.0136 - accuracy: 0.0000e+00 - val_loss: 2.6912 - val_mean_absolute_error: 0.9991 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 5/20
98/98 [==============================] - 26s 270ms/step - los

UnknownError: Graph execution error:

RuntimeError: generator raised StopIteration
Traceback (most recent call last):

  File "d:\Anaconda\envs\tf\lib\site-packages\keras\engine\data_adapter.py", line 1050, in generator_fn
    yield x[i]

  File "d:\Github\TesiRacingGame\AnacondaScripts\newDB\Generator.py", line 132, in __getitem__
    x,y=next(self.generator)

StopIteration


The above exception was the direct cause of the following exception:


Traceback (most recent call last):

  File "d:\Anaconda\envs\tf\lib\site-packages\tensorflow\python\ops\script_ops.py", line 271, in __call__
    ret = func(*args)

  File "d:\Anaconda\envs\tf\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "d:\Anaconda\envs\tf\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 1035, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "d:\Anaconda\envs\tf\lib\site-packages\keras\engin

In [82]:
test=0
for batch in gen_val:
    print(test)
    test+=1
test

48

In [56]:
test[1].shape

IndexError: list index out of range

In [93]:
y_pred = lstm_ph.predict(gen_test)


47/47 [==============================] - 7s 151ms/step


In [94]:
y_pred

array([[ 1.4616523 , -0.7965714 , -2.364098  ],
       [ 1.4196657 , -0.72882974, -2.1959066 ],
       [ 1.3491808 , -0.6210692 , -2.2700336 ],
       ...,
       [ 0.10450363,  0.48482472, -0.6983368 ],
       [ 0.10220872,  0.48504275, -0.7115003 ],
       [ 0.09761263,  0.49852407, -0.72805303]], dtype=float32)

In [95]:
ev

[2.1729495525360107, 0.8310683965682983, 0.0]

In [27]:
lr =1e-3
units=512
cells=8
normalization=False
name=f"LSTM{cells}_{units}_{lr}_DISCARD_{DISCARD}_DELTA"
lstm_ph,history_lstm_ph=model_definitions.LSTM_Train(gen_train,gen_val,learning_rate=lr,patience=50,name=name,path="logs",
                                                     epoch=100, dropout=0.2, cells=cells,units=units, normalization=normalization,verbose=1)

ev=lstm_ph.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")
tf.saved_model.save(lstm_ph,f'saved_model/{name}')

INFO:tensorflow:Assets written to: saved_model/LSTM8_512_0.001_DISCARD_19_DELTA\assets


INFO:tensorflow:Assets written to: saved_model/LSTM8_512_0.001_DISCARD_19_DELTA\assets


In [29]:
tf.saved_model.save(lstm_ph,f'saved_model/{name}')

INFO:tensorflow:Assets written to: saved_model/LSTM4_128_0.001_DISCARD_19_DELTA\assets


INFO:tensorflow:Assets written to: saved_model/LSTM4_128_0.001_DISCARD_19_DELTA\assets


In [12]:
model=model_definitions.MLP_Model(32)
model.load_weights("saved_model\MLP0001_PosAssoluta")

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 140)               0         
                                                                 
 dense (Dense)               (None, 32)                4512      
                                                                 
 dense_1 (Dense)             (None, 3)                 99        
                                                                 
Total params: 4,611
Trainable params: 4,611
Non-trainable params: 0
_________________________________________________________________


In [14]:
zero=gen_test[0]
toP=zero[0][0]
pred=zero[1][0]
print(toP)
print(pred)
model.predict(toP)

tf.Tensor(
[[-2.156165e+01 -1.154109e+01  3.978183e-01  1.900000e+01  2.000000e+01
   2.002995e-01  5.461393e-01]
 [-2.191261e+01 -1.069794e+01  9.277464e-01  1.900000e+01  2.000000e+01
   1.828646e-01  5.285156e-01]
 [-2.236745e+01 -9.373469e+00  1.181234e+00  1.900000e+01  2.000000e+01
   1.650993e-01  5.127604e-01]
 [-2.271277e+01 -8.215466e+00  8.408008e-01  1.900000e+01  2.000000e+01
   1.469889e-01  4.992383e-01]
 [-2.317203e+01 -7.699648e+00  2.726782e-01  1.900000e+01  2.000000e+01
   1.285677e-01  4.866276e-01]
 [-2.354665e+01 -7.331343e+00  8.814032e-02  1.900000e+01  2.000000e+01
   1.098258e-01  4.747331e-01]
 [-2.385743e+01 -7.165668e+00  3.089963e-02  1.900000e+01  2.000000e+01
   9.082031e-02  4.631966e-01]
 [-2.413132e+01 -7.114067e+00  1.184550e-02  1.900000e+01  2.000000e+01
   7.158691e-02  4.517969e-01]
 [-2.438515e+01 -7.120355e+00  4.894121e-03  1.900000e+01  2.000000e+01
   5.214518e-02  4.404102e-01]
 [-2.460253e+01 -6.905964e+00  1.876919e-01  1.900000e+01  2.0

ValueError: in user code:

    File "d:\Anaconda\envs\tf\lib\site-packages\keras\engine\training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "d:\Anaconda\envs\tf\lib\site-packages\keras\engine\training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "d:\Anaconda\envs\tf\lib\site-packages\keras\engine\training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "d:\Anaconda\envs\tf\lib\site-packages\keras\engine\training.py", line 1983, in predict_step
        return self(x, training=False)
    File "d:\Anaconda\envs\tf\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "d:\Anaconda\envs\tf\lib\site-packages\keras\engine\input_spec.py", line 295, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 of layer "sequential" is 

In [ ]:
import tf2onnx
import onnx
input_signature=(tf.TensorSpec([None,SEQUENCE_LENGTH,FEATURES],tf.float32,name='x'))
onnx_model,_=tf2onnx.converter.from_keras(model,input_signature,opset=9)

In [49]:
races=united["RACE"].max()
print(races*0.5)
training=united.loc[united['RACE'] <= races*0.5]
training.tail(10)

83.5


,X,Z,VEL_X,VEL_Z,ROT,ANG_VEL_Y,ACC_X,ACC_Z,TILE,TILE_IND,X_RELATIVE,Z_RELATIVE,TIME,RACE,GROUP
5051758,-5.32727,2.571900,4.838010,5.90930,13.9886,0.747973,27.840900,24.523110,4,32,0.456929,0.080146,0.2,83,9
5051759,-4.23840,3.754450,4.522620,4.17718,14.0799,-0.201847,-13.900470,-23.734710,4,32,0.400417,0.130206,0.2,83,9
5051760,-3.61328,4.332640,1.717370,2.05974,4.8036,-1.075305,-18.862057,-9.560299,4,32,0.352240,0.187974,0.2,83,9
5051761,-3.38379,4.573670,1.456980,0.93513,2.5532,0.547570,12.505337,0.734615,4,32,0.307122,0.248957,0.2,83,9
5051762,-3.03601,4.786750,1.265280,0.82554,3.1718,-0.565750,-4.297250,-2.067471,4,32,0.266642,0.312780,0.2,83,9
5051763,-3.02600,4.787650,0.228000,-0.06486,-0.8624,0.594987,2.488040,-1.159763,4,32,0.226296,0.376615,0.2,83,9
5051764,-2.67359,4.945070,2.600840,1.12468,7.7518,0.070049,5.761860,3.362847,4,32,0.190648,0.442550,0.2,83,9
5051765,-1.90893,5.190311,4.206075,1.07436,9.8041,-0.011295,-8.786300,-4.183674,16,0,0.165195,-0.476492,0.2,83,9
5051766,-1.24964,5.344787,2.852189,0.50855,6.1715,0.238537,11.289770,-2.462674,16,0,0.148534,-0.333965,0.2,83,9
5051767,-0.57885,5.431946,3.084546,0.64175,5.5514,-0.873222,-14.921353,2.689744,16,0,0.140815,-0.189113,0.2,83,9


In [25]:
t=df_train["ROT"].idxmax()
df_train.iloc[t-2:t+3]

,X,Z,VEL_X,VEL_Z,ROT,ANG_VEL_Y,ACC_X,ACC_Z,TILE,TILE_IND,X_RELATIVE,Z_RELATIVE,TIME,RACE,GROUP
2470682,0.00270,-4.962400,0.123759,-0.813940,-0.0700,0.055793,-1.702185,-0.837422,17,22,-0.459295,-0.110540,0.2,5,9
2470683,0.00250,-5.130860,-0.017524,-0.787650,0.0002,0.000706,0.121586,0.341511,17,22,-0.459261,-0.178952,0.2,5,9
2470684,-343.65538,23.621341,-0.006691,26.007860,179.8327,-0.145945,0.091397,3.962040,16,0,-0.041333,-0.228001,0.2,5,9
2470685,-0.00781,0.513184,-0.092801,4.798755,-0.9364,0.148455,-0.496246,25.744220,16,0,-0.041437,-0.214316,0.2,5,9
2470686,-0.03454,1.323730,-0.101864,3.123408,-0.3048,-0.017739,0.488042,-7.024400,16,0,-0.041898,-0.179017,0.2,5,9
